In [26]:
import os

In [27]:
%pwd

'c:\\Users\\vvlachop\\Documents\\Text-Summarizer'

In [28]:
os.chdir("c:\\Users\\vvlachop\\Documents\\Text-Summarizer")

In [29]:
%pwd

'c:\\Users\\vvlachop\\Documents\\Text-Summarizer'

In [30]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [31]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [33]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [34]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
        output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=50,
        per_device_train_batch_size=1, per_device_eval_batch_size=1,
        weight_decay=0.01, logging_steps=10,
        evaluation_strategy='steps', eval_steps=50, save_steps=1e6,
        gradient_accumulation_steps=16, max_steps=50  # Add max_steps to limit training to 50 iterations
)

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])

        trainer.train()

        


In [35]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-28 11:25:50,541: INFO :common: yaml file: config\config.yaml loaded successfully]
[2023-12-28 11:25:50,543: INFO :common: yaml file: params.yaml loaded successfully]
[2023-12-28 11:25:50,545: INFO :common: created directory at: artifacts]
[2023-12-28 11:25:50,546: INFO :common: created directory at: artifacts/model_trainer]


  0%|          | 0/920 [07:01<?, ?it/s]
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|██        | 10/50 [08:10<31:32, 47.32s/it]

{'loss': 3.1621, 'learning_rate': 1e-05, 'epoch': 0.01}


 40%|████      | 20/50 [16:19<24:58, 49.96s/it]

{'loss': 2.9691, 'learning_rate': 2e-05, 'epoch': 0.02}


...

{'loss': 2.1439, 'learning_rate': 4e-05, 'epoch': 0.04}


100%|██████████| 50/50 [42:09<00:00, 53.21s/it]

{'loss': 1.9429, 'learning_rate': 0.0, 'epoch': 0.05}


                                               
100%|██████████| 50/50 [52:50<00:00, 63.42s/it]  

{'eval_loss': 1.763014554977417, 'eval_runtime': 640.9463, 'eval_samples_per_second': 1.276, 'eval_steps_per_second': 1.276, 'epoch': 0.05}
{'train_runtime': 3170.7738, 'train_samples_per_second': 0.252, 'train_steps_per_second': 0.016, 'train_loss': 2.534224548339844, 'epoch': 0.05}
